# Sampling I: direct and importance sampling


## Content
- Monte Carlo integration methods

## Remember jupyter notebooks
- To run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>.
- To get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## "Area under a curve" via direct Monte Carlo sampling

Let us define `f1(x, mu, sigma)` which evaluates

$$f1(x, \mu, \sigma) = \frac{1}{\sqrt{2\pi}\sigma} \exp\left( -\frac{(x - \mu)^2}{2\sigma^2} \right)$$

as well as `f2(x)` and  `f3(x)` which evaluate superpositions of `f1` calls for different centers and scales:

In [ ]:
def f1(x, mu=0.0, sigma=1.0):
    gauss_curve = np.exp(-(x - mu)**2 / (2.0 * sigma**2))
    area = np.sqrt(2.0 * np.pi) * sigma
    return gauss_curve / area


x = np.linspace(-5, 5, 1000)

fig, ax = plt.subplots()
ax.plot(x, f1(x))
ax.fill_between(x, 0, f1(x), alpha=0.3)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$f1(x)$')
fig.tight_layout()

In [ ]:
def integrate_mc(func, xmin, xmax, size):
    x = np.random.uniform(low=xmin, high=xmax, size=size)
    f = func(x)
    ymax = f.max()
    x_width = xmax - xmin
    y = np.random.uniform(low=0, high=ymax, size=size)
    n = np.sum(y < f)
    area = x_width * ymax
    ratio = n / size
    return area * ratio, area, ratio


integrate_mc(f1, -5, 5, 100000)

In [ ]:
def f2(x):
    return 0.5 * (f1(x, mu=-2, sigma=0.5) + f1(x, mu=2, sigma=0.5))


x = np.linspace(-5, 5, 1000)

fig, ax = plt.subplots()
ax.plot(x, f2(x))
ax.fill_between(x, 0, f2(x), alpha=0.3)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$f2(x)$')
fig.tight_layout()

integrate_mc(f2, -5, 5, 100000)

In [ ]:
def f3(x):
    return 0.5 * (f1(x, mu=-4.5, sigma=0.05) + f1(x, mu=4.5, sigma=0.05))


x = np.linspace(-5, 5, 1000)

fig, ax = plt.subplots()
ax.plot(x, f3(x))
ax.fill_between(x, 0, f3(x), alpha=0.3)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$f3(x)$')
fig.tight_layout()

integrate_mc(f3, -5, 5, 100000)

### Exercise:

How do the standard deviations of the integrals of the three functions scale with the number of evaluation points?

In [ ]:
n_values = [100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000, 100000]

fig, ax = plt.subplots(figsize=(10, 6))

for i, f in enumerate((f1, f2, f3)):
    data = None
    ax.plot(n_values, data, '-o', linewidth=2, color=f'C{i}', label=f'f{i + 1}')
    ax.fill_between(n_values, 0, data, facecolor=f'C{i}', alpha=0.3)

ax.semilogx()
ax.legend()
ax.set_xlabel(r'$N$')
ax.set_ylabel(r'$\sigma$')
fig.tight_layout()

Why does `f3` scale so much worde than the two other functions? We can understand this problem by looking at the ratios of the areas under the curves versus the total integration/sampling areas:

In [ ]:
def integrate_mc_show(func, xmin, xmax, size, ax=None):
    x = np.random.uniform(low=xmin, high=xmax, size=size)
    f = func(x)
    ymax = f.max()
    x_width = xmax - xmin
    y = np.random.uniform(low=0, high=ymax, size=size)
    n = np.sum(y < f)
    area = x_width * ymax
    ratio = n / size
    if ax is None:
        _, ax = plt.subplots()
    ax.plot([xmin] * 2, [0, ymax], color='C3')
    ax.plot([xmax] * 2, [0, ymax], color='C3')
    ax.plot([xmin, xmax], [0] * 2, color='C3')
    ax.plot([xmin, xmax], [ymax] * 2, color='C3')
    ax.scatter(x, y, c=y < f, s=0.1)
    ax.set_xlabel(r'$x$')
    ax.set_ylabel(r'$y$')
    return area * ratio, area, ratio, ax


fig, axes = plt.subplots(1, 3, figsize=(10, 3))
for i, (ax, f) in enumerate(zip(axes.flat, (f1, f2, f3))):
    _, _, ratio, _ = integrate_mc_show(f, -5, 5, 10000, ax=ax)
    ax.set_title(f'f{i + 1}, {ratio}')
fig.tight_layout()

## Computing expectation values

In [ ]:
def harmonic_potential(x):
    return np.power(x, 2)


x = np.linspace(-5, 5, 1000)

fig, ax = plt.subplots()
ax.plot(x, harmonic_potential(x))
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$x^2$')
fig.tight_layout()

We define a potential $\phi(x) = x^2$ as shown above.
According to statistical mechanics, the probablility to observe a
position/configuration $x$ is proportional to the Boltzmann weight

$$
p(x) \propto e^{-\beta\phi(x)} = e^{-\frac{x^2}{\mathrm{k}_\mathrm{B}T}}\,,
\quad \mathrm{with} \; \beta^{-1}=\mathrm{k}_\mathrm{B}T
$$

$\beta$ is called an inverse temperature and has the dimension of an inverse energy.

We further define a uniform grid

$$x_i = x_0 + ih \,,\; i=0,\dots,n$$

and a sequence of indicator functions

$$\chi_i(x) = \begin{cases} 1, & x_i \leq x < x_{i+1} \\ 0, & \mathrm{else} \end{cases}\,, \quad i=0,\dots,n-1.$$

Let us use our previous approach to compute a histogram of positions
to approximate the stationary distribution $\pi(x)$:

In [ ]:
def chi(x, xmin, xmax):
    return np.logical_and(xmin <= x, x < xmax)


positions = np.random.uniform(low=-5, high=5, size=100000)
edges = np.linspace(-5, 5, 31)
centers = (edges[:-1] + edges[1:]) / 2

histogram = [np.sum(chi(positions, x, y)) / positions.size
             for x, y in zip(edges[:-1], edges[1:])]

fig, ax = plt.subplots()
ax.bar(centers, [h for h in histogram], (edges[1] - edges[0]) * 0.9)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$\frac{1}{N}\sum_{n=1}^N \chi_i(x_n)$')
fig.tight_layout()

This does not look right!
The naive averaging yields an approximately constant distribution
which is not compatible with the expected Boltzmann distribution.

The reason is simply that the naive average does not compute a **weighted** expectation:

$$\frac{1}{N}\sum_{n=1}^N \chi_i(x_n) \neq \mathbb{E}_p \left[\chi_i \right]$$

Instead, we need to compute 

$$\sum_{n=1}^N \chi_i(x_n) p(x_n)\,, \quad \mathrm{with} \; \sum_{n=1}^N p(x_n)=1:$$

In [ ]:
def pairs(iterable):
    for x, y in zip(iterable[:-1], iterable[1:]):
        yield x, y


weights = np.exp(-harmonic_potential(positions) * 1.0)
weights /= weights.sum()

histogram2 = [np.sum(weights[chi(positions, x, y)])
              for x, y in pairs(edges)]

fig, ax = plt.subplots()
ax.bar(
    centers,
    [h for h in histogram],
    (edges[1] - edges[0]) * 0.9,
    label=r'$\frac{1}{N}\sum_{n=1}^N \chi_i(x_n)$')
ax.bar(
    centers,
    [h for h in histogram2],
    (edges[1] - edges[0]) * 0.8,
    label=r'$\sum_{n=1}^N \chi_i(x_n)p(x_n)$')
ax.set_xlabel(r'$x$')
ax.legend()
fig.tight_layout()

With this correction, we get the expected distribution, but we are still wasting computational effort.

A better strategy is to exploit that we can directly sample from the Boltzmann distribution in our case of a harmonic potential. We just have to replace the way we draw random numbers:

In [ ]:
positions = np.random.normal(size=100000)
edges = np.linspace(-5, 5, 31)
centers = (edges[:-1] + edges[1:]) / 2

histogram = [np.sum(chi(positions, x, y)) / positions.size
             for x, y in zip(edges[:-1], edges[1:])]

fig, ax = plt.subplots()
ax.bar(centers, [h for h in histogram], (edges[1] - edges[0]) * 0.9)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$\frac{1}{N}\sum_{n=1}^N \chi_i(x_n)$, $x_n\sim\mathcal{N}(0, 1)$')
fig.tight_layout()

If we can sample directly from $p$, we can compute expectations without reweighting.

In most cases, however, we cannot **directly** sample from $p$ and we have to resort to more complicated procedures,
e.g., sampling a Markov chain of positions/configurations using the Metropolis scheme
where we accept a transition from $x$ to $y$ with the conditional acceptance probablility

$$\mathbb{A}(y|x) = \min\left\{1, \exp\left(\beta(\phi(x) - \phi(y))\right)\right\}.$$

### Exercise:

Complete the following function to implement the Metropolis sampling algorithm.

In [ ]:
def mmc(potential, size, x_init=0.0, beta=1.0, step=0.5):
    from random import uniform
    x, u = np.zeros(size), np.zeros(size)
    x[0] = x_init
    u[0] = potential(x_init)
    for i in range(1, size):
        pass
    return x, u


positions, energies = mmc(harmonic_potential, 10000)
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(positions[:400], label=r'positions')
ax.plot(energies[:400], label=r'energies')
ax.set_xlabel(r'$t$ / steps')
ax.legend()
fig.tight_layout()

In [ ]:
edges = np.linspace(positions.min(), positions.max(), 31)
centers = (edges[:-1] + edges[1:]) / 2

histogram = [np.sum(chi(positions, x, y)) / positions.size
             for x, y in zip(edges[:-1], edges[1:])]

fig, ax = plt.subplots()
ax.bar(centers, [h for h in histogram], (edges[1] - edges[0]) * 0.9)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$\frac{1}{N}\sum_{n=1}^N \chi_i(x_n)$, $x_n\sim e^{-\beta\phi(x_n)}$')
fig.tight_layout()